In [4]:
!pip install -q botometer

In [0]:
from openpyxl import load_workbook, Workbook
from botometer import Botometer
import pandas as pd
import random

In [11]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving config.csv to config.csv
User uploaded file "config.csv" with length 45 bytes


In [0]:
def print_dataset(dataset):
  for row in twitter_dataset.itertuples():
    print(row)
    # print(row[1])

def sort_dataset(dataset, col_names, ascending_vals):
  # sorting data frame by name
  dataset.sort_values(col_names, axis = 0, ascending = ascending_vals, 
                      inplace = True, na_position ='last')
  
def filter_dataset(dataset, col_name, col_value):
  # filtering data frame by name
  col_name = col_name.lower()
  if col_name in dataset.columns:
    return dataset[dataset[col_name].str.lower().isin([col_value.casefold()])]
  return dataset

In [0]:
def set_users_bot_prob(users_list):
  global users_bot_prob
  for user in users_list:
    if user not in users_bot_prob:
      bot_prob = random.uniform(0, 1)
      if bot_prob >= 0.6:
        users_bot_prob[user] = bot_prob

def discard_tweet_bots(dataset):
  global users_bot_prob
  set_users_bot_prob(dataset['author id'].unique())
  print(users_bot_prob)
  return dataset[~dataset['author id'].isin(users_bot_prob)]


In [61]:
sentiment_types = {'NEGATIVE': 1, 'NEUTRAL': 2, 'POSITIVE': 3}
users_bot_prob = {}
config_file2 = pd.read_csv('config.csv', header=None)
data_file = pd.ExcelFile('sol_sample_uhrcn.xlsx')

# Creating dataframe from Twitter data sheet
twitter_dataset = data_file.parse(data_file.sheet_names[0])
twitter_dataset.columns = twitter_dataset.columns.str.lower()
print('Initial shape:', twitter_dataset.shape)
# print(twitter_dataset.columns)

twitter_dataset = discard_tweet_bots(twitter_dataset)
print('NO BOTS shape:', twitter_dataset.shape)
print('TOTAL BOTS:', len(users_bot_prob))
# Read the sorting commands from config file
list_sort_cols = []
list_sort_as_des = []
for command in config_file2.itertuples():
  if command[1].lower() == 'sortby':
    col_name = command[2].lower()
    if col_name in twitter_dataset.columns:
      list_sort_cols.append(col_name)
      list_sort_as_des.append(command[3].lower() == 'a')

if list_sort_cols:
  sort_dataset(twitter_dataset, list_sort_cols, list_sort_as_des)

# Process commands from config file
for command in config_file2.itertuples():
  if command[1].lower() != 'sortby':
    if command[1].lower() == 'filterby':
      twitter_dataset = filter_dataset(twitter_dataset, command[2], command[3])

# print(twitter_dataset.shape)

# Output file
twitter_dataset.to_excel("optimised_file.xlsx")


Initial shape: (99, 33)
{'simonilli': 0.9339640699836466, 'carlogubi': 0.7408734856442469, 'La_LadyGabry': 0.7797942354804903, 'LESOVICI': 0.922453972312156, 'piemarcy': 0.7125098160463199, 'Marinotoma': 0.7451761068139757, 'Corriere': 0.9700216654089461, 'GiovanniMario12': 0.9060781778001548, 'Noviolenzadonne': 0.8329332553269138, 'GenovaOn': 0.7450967264306045, 'roman72gfr': 0.7459087703389269, 'floracappelluti': 0.999968297229314, 'Paternoster85': 0.7628833570927634, 'godzillante': 0.8172811302608701, 'ElenaCaracozza': 0.6544459230411622, 'AFoccardi': 0.9719630492558312, 'MariaRitaDAdamo': 0.8139290869544622, 'giovannella58': 0.7790201201734939, 'repubblica': 0.9438347331836061, 'LNocoro': 0.8367861255854946, 'voltes48': 0.7431933268393094, 'nelloscavo': 0.6155817131918371, 'ghezzi_grazia': 0.7974602741773402, 'Maniheli': 0.7993243834948471, 'thenightfly85': 0.9013882130772777}
NO BOTS shape: (61, 33)
TOTAL BOTS: 25


In [0]:
twitter_dataset